In [1]:
pip install mysql-connector-python

   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   -- ------------------------------------- 1.0/16.1 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 2.1/16.1 MB 5.6 MB/s eta 0:00:03
   -------- ------------------------------- 3.4/16.1 MB 5.8 MB/s eta 0:00:03
   ------------ --------------------------- 5.0/16.1 MB 6.0 MB/s eta 0:00:02
   --------------- ------------------------ 6.0/16.1 MB 6.0 MB/s eta 0:00:02
   ----------------- ---------------------- 7.1/16.1 MB 5.8 MB/s eta 0:00:02
   ------------------- -------------------- 7.9/16.1 MB 5.4 MB/s eta 0:00:02
   -------------------- ------------------- 8.4/16.1 MB 5.2 MB/s eta 0:00:02
   --------------------- ------------------ 8.7/16.1 MB 4.8 MB/s eta 0:00:02
   ---------------------- ----------------- 9.2/16.1 MB 4.3 MB/s eta 0:00:02
   ----------------------- ---------------- 9.4/16.1 MB 4.2 MB/s eta 0:00:02
   ------------------------ --------------- 9.7/16.1 MB 4.1 MB/s eta 0:00:02
   ---

In [6]:
import re
import mysql.connector

# Connect to MySQL and prepare the database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="####"
)
cursor = conn.cursor()

# Drop and recreate the database to ensure a clean slate
cursor.execute("DROP DATABASE IF EXISTS my_temp_db;")
cursor.execute("CREATE DATABASE my_temp_db;")
print("Database 'my_temp_db' created successfully.")

# Switch to using the newly created database
conn.database = 'my_temp_db'

# Step 1: Drop tables if they exist and create fresh tables
with open('C:\\Users\\Ayomide Joe\\Documents\\VSCode\\NaijaMed_Q-A_Dataset\\create_table.sql', 'r', encoding='utf-8') as f:
    create_tables_script = "SET foreign_key_checks = 0;\n"  # Disable foreign key checks
    for line in f:
        if line.strip().upper().startswith("CREATE TABLE"):
            table_name = line.strip().split()[2]  # Extract the table name
            create_tables_script += f"DROP TABLE IF EXISTS {table_name};\n"
        create_tables_script += line
    create_tables_script += "\nSET foreign_key_checks = 1;"  # Re-enable foreign key checks

try:
    for result in cursor.execute(create_tables_script, multi=True):
        pass
    print("Tables created successfully after dropping existing ones.")
except Exception as e:
    print(f"Error during table creation: {e}")

# Step 2: Prepare and execute INSERT statements by removing database prefix
with open('C:\\Users\\Ayomide Joe\\Documents\\VSCode\\NaijaMed_Q-A_Dataset\\ebdb_threads.sql', 'r', encoding='utf-8') as f:
    insert_data_script = f.read()
    
    # Remove references to `ebdb.`
    insert_data_script = re.sub(r'INSERT INTO\s+ebdb\.', 'INSERT INTO ', insert_data_script)

    insert_data_script = f"SET foreign_key_checks = 0;\n{insert_data_script}\nSET foreign_key_checks = 1;"

try:
    for result in cursor.execute(insert_data_script, multi=True):
        pass
    print("Data inserted successfully with foreign key checks disabled.")
except Exception as e:
    print(f"Error during data insertion: {e}")

conn.commit()
conn.close()


Database 'my_temp_db' created successfully.
Tables created successfully after dropping existing ones.
Data inserted successfully with foreign key checks disabled.


In [7]:
# Connect to your database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="####",
    database="my_temp_db"
)
cursor = conn.cursor()

# Execute a query to show all tables
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()

# Print each table name
print("Tables in my_temp_db:")
for table in tables:
    print(table[0])

conn.close()


Tables in my_temp_db:
threads


In [9]:
import pandas as pd
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="####",
    database="my_temp_db"
)

# Load data from a table into a DataFrame
df = pd.read_sql("SELECT * FROM threads", conn)  # Replace 'your_table' with the actual table name
print(df.head())

conn.close()

C:\Users\Ayomide Joe\AppData\Local\Temp\ipykernel_112264\3694312356.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM threads", conn)  # Replace 'your_table' with the actual table name


   id  category_id  user_id  \
0   1            3        6   
1   2           10        9   
2   3            8       11   
3   4            5       11   
4   5           10       11   

                                             content  anonymous  \
0  All of a sudden, I have been having episodes o...          1   
1  Is it a must to get a crown after my my root c...          0   
2  I usually have this itch all over my body anyt...          1   
3  i have this whitish discharge and itch in my p...          1   
4  i have a friend who has been battling with mou...          1   

           created_at          updated_at deleted_at  trending  pinned  \
0 2018-06-30 19:48:14 2019-07-07 13:56:39        NaT         0       0   
1 2018-07-11 00:17:44 2019-07-07 13:57:03        NaT         0       0   
2 2018-07-14 01:17:05 2019-07-07 15:59:40        NaT         0       0   
3 2018-07-14 01:19:56 2019-07-07 16:00:03        NaT         0       0   
4 2018-07-14 01:23:42 2019-07-07 16:00:2

In [10]:
df.head()

,id,category_id,user_id,content,anonymous,created_at,updated_at,deleted_at,trending,pinned,pinning_order
0,1,3,6,"All of a sudden, I have been having episodes o...",1,2018-06-30 19:48:14,2019-07-07 13:56:39,NaT,0,0,None
1,2,10,9,Is it a must to get a crown after my my root c...,0,2018-07-11 00:17:44,2019-07-07 13:57:03,NaT,0,0,None
2,3,8,11,I usually have this itch all over my body anyt...,1,2018-07-14 01:17:05,2019-07-07 15:59:40,NaT,0,0,None
3,4,5,11,i have this whitish discharge and itch in my p...,1,2018-07-14 01:19:56,2019-07-07 16:00:03,NaT,0,0,None
4,5,10,11,i have a friend who has been battling with mou...,1,2018-07-14 01:23:42,2019-07-07 16:00:21,NaT,0,0,None


In [12]:
csv_file_path = f"{table_name}.csv"  
df.to_csv(csv_file_path, index=False)  # index=False to avoid saving row numbers